In [1]:
from langgraph.graph import StateGraph, END

# to construct agent state
from typing import TypedDict, Annotated
import operator

# to construct human, ai and system messages
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage

# llm 
from langchain_ollama import ChatOllama

# tool
from langchain_community.tools.tavily_search import TavilySearchResults

In [2]:
# old
import os
TAVILY_API_KEY = "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"
os.environ["TAVILY_API_KEY"] = TAVILY_API_KEY

In [3]:
# old
from langgraph.checkpoint.sqlite import SqliteSaver
memory = SqliteSaver.from_conn_string(":memory:")

In [4]:
from uuid import uuid4
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, AIMessage

"""
In previous examples we've annotated the `messages` state key
with the default `operator.add` or `+` reducer, which always
appends new messages to the end of the existing messages array.

Now, to support replacing existing messages, we annotate the
`messages` key with a customer reducer function, which replaces
messages with the same `id`, and appends them otherwise.
"""
def reduce_messages(left: list[AnyMessage], right: list[AnyMessage]) -> list[AnyMessage]:
    # assign ids to messages that don't have them
    for message in right:
        if not message.id:
            message.id = str(uuid4())
    # merge the new messages with the existing messages
    merged = left.copy()
    for message in right:
        for i, existing in enumerate(merged):
            # replace any existing messages with the same id
            if existing.id == message.id:
                merged[i] = message
                break
        else:
            # append any new messages to the end
            merged.append(message)
    return merged

class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], reduce_messages]

In [5]:
#old [modified]
class Agent:
    def __init__(self, model, tools, system="", checkpointer=None):
        self.system = system
        graph = StateGraph(AgentState)
        graph.add_node("llm", self.call_openai)
        graph.add_node("action", self.take_action)
        graph.add_conditional_edges("llm", self.exists_action, {True: "action", False: END})
        graph.add_edge("action", "llm")
        graph.set_entry_point("llm")
        self.graph = graph.compile(
            checkpointer=checkpointer,
            interrupt_before=["action"] ### added for manual approval
        )
        self.tools = {t.name: t for t in tools}
        self.model = model.bind_tools(tools) ### called before action

    def call_openai(self, state: AgentState):
        messages = state['messages']
        if self.system:
            messages = [SystemMessage(content=self.system)] + messages
        message = self.model.invoke(messages)
        return {'messages': [message]}

    def exists_action(self, state: AgentState):
        print(state)
        result = state['messages'][-1]
        return len(result.tool_calls) > 0

    def take_action(self, state: AgentState):
        tool_calls = state['messages'][-1].tool_calls
        results = []
        for t in tool_calls:
            print(f"Calling: {t}")
            result = self.tools[t['name']].invoke(t['args'])
            results.append(ToolMessage(tool_call_id=t['id'], name=t['name'], content=str(result)))
        print("Back to the model!")
        return {'messages': results}

In [6]:
prompt = """You are a smart research assistant. Use the search engine to look up information. \
You are allowed to make multiple calls (either together or in sequence). \
Only look up information when you are sure of what you want. \
If you need to look up some information before asking a follow up question, you are allowed to do that!
"""

model = ChatOllama(model="llama3.1")

tool = TavilySearchResults(max_results=2)

abot = Agent(model, [tool], system=prompt, checkpointer=memory)

ValidationError: 1 validation error for CompiledStateGraph
checkpointer
  instance of BaseCheckpointSaver expected (type=type_error.arbitrary_type; expected_arbitrary_type=BaseCheckpointSaver)

In [7]:
messages = [HumanMessage(content="What is the weather in sf?")]
thread = {"configurable": {"thread_id": "1"}}
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v['messages'])

{'messages': [HumanMessage(content='What is the weather in sf?', id='6da9c792-36d4-417e-b401-7cf57bf1e726'), AIMessage(content='', response_metadata={'model': 'llama3.1', 'created_at': '2024-08-12T12:15:04.38621Z', 'message': {'role': 'assistant', 'content': '', 'tool_calls': [{'function': {'name': 'tavily_search_results_json', 'arguments': {'query': 'sf weather'}}}]}, 'done_reason': 'stop', 'done': True, 'total_duration': 31376375300, 'load_duration': 21196892100, 'prompt_eval_count': 249, 'prompt_eval_duration': 2830293000, 'eval_count': 22, 'eval_duration': 6540612000}, id='run-95302bdc-0fe1-4957-8a25-dae431504340-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'sf weather'}, 'id': '6a88d71a-5284-4886-b585-68783f051089', 'type': 'tool_call'}], usage_metadata={'input_tokens': 249, 'output_tokens': 22, 'total_tokens': 271})]}
[AIMessage(content='', response_metadata={'model': 'llama3.1', 'created_at': '2024-08-12T12:15:04.38621Z', 'message': {'role': 'assistan

get current state

In [8]:
abot.graph.get_state(thread)

StateSnapshot(values={'messages': [HumanMessage(content='What is the weather in sf?', id='6da9c792-36d4-417e-b401-7cf57bf1e726'), AIMessage(content='', response_metadata={'model': 'llama3.1', 'created_at': '2024-08-12T12:15:04.38621Z', 'message': {'role': 'assistant', 'content': '', 'tool_calls': [{'function': {'name': 'tavily_search_results_json', 'arguments': {'query': 'sf weather'}}}]}, 'done_reason': 'stop', 'done': True, 'total_duration': 31376375300, 'load_duration': 21196892100, 'prompt_eval_count': 249, 'prompt_eval_duration': 2830293000, 'eval_count': 22, 'eval_duration': 6540612000}, id='run-95302bdc-0fe1-4957-8a25-dae431504340-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'sf weather'}, 'id': '6a88d71a-5284-4886-b585-68783f051089', 'type': 'tool_call'}], usage_metadata={'input_tokens': 249, 'output_tokens': 22, 'total_tokens': 271})]}, next=('action',), config={'configurable': {'thread_id': '1', 'thread_ts': '1ef58a48-81b4-679c-8001-8138eea2faa2'}}

get the next node

In [9]:
abot.graph.get_state(thread).next

('action',)

continue after interrupt

In [10]:
for event in abot.graph.stream(None, thread):
    for v in event.values():
        print(v)

Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'sf weather'}, 'id': '6a88d71a-5284-4886-b585-68783f051089', 'type': 'tool_call'}
Back to the model!
{'messages': [ToolMessage(content='[{\'url\': \'https://www.weatherapi.com/\', \'content\': "{\'location\': {\'name\': \'San Francisco\', \'region\': \'California\', \'country\': \'United States of America\', \'lat\': 37.78, \'lon\': -122.42, \'tz_id\': \'America/Los_Angeles\', \'localtime_epoch\': 1723464778, \'localtime\': \'2024-08-12 05:12\'}, \'current\': {\'last_updated_epoch\': 1723464000, \'last_updated\': \'2024-08-12 05:00\', \'temp_c\': 13.2, \'temp_f\': 55.8, \'is_day\': 0, \'condition\': {\'text\': \'Partly Cloudy\', \'icon\': \'//cdn.weatherapi.com/weather/64x64/night/116.png\', \'code\': 1003}, \'wind_mph\': 9.2, \'wind_kph\': 14.8, \'wind_degree\': 240, \'wind_dir\': \'WSW\', \'pressure_mb\': 1014.0, \'pressure_in\': 29.95, \'precip_mm\': 0.0, \'precip_in\': 0.0, \'humidity\': 89, \'cloud\': 47, \'feelslik

add user input prompt to continue

In [ ]:
messages = [HumanMessage("Whats the weather in LA?")]
thread = {"configurable": {"thread_id": "2"}}
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v)
while abot.graph.get_state(thread).next:
    print("\n", abot.graph.get_state(thread),"\n")
    _input = input("proceed?")
    if _input != "y":
        print("aborting")
        break
    for event in abot.graph.stream(None, thread):
        for v in event.values():
            print(v)

# Modify state

In [ ]:
messages = [HumanMessage("Whats the weather in LA?")]
thread = {"configurable": {"thread_id": "3"}}
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v)

In [ ]:
abot.graph.get_state(thread)

In [ ]:
current_values = abot.graph.get_state(thread)

In [ ]:
current_values.values['messages'][-1]

In [ ]:
current_values.values['messages'][-1].tool_calls

In [ ]:
_id = current_values.values['messages'][-1].tool_calls[0]['id']
current_values.values['messages'][-1].tool_calls = [
    {'name': 'tavily_search_results_json',
  'args': {'query': 'current weather in Louisiana'},
  'id': _id}
]

In [ ]:
abot.graph.update_state(thread, current_values.values)

In [ ]:
abot.graph.get_state(thread)

In [ ]:
for event in abot.graph.stream(None, thread):
    for v in event.values():
        print(v)

# Time Travel

In [11]:
states = []
for state in abot.graph.get_state_history(thread):
    print(state)
    print('--')
    states.append(state)

StateSnapshot(values={'messages': [HumanMessage(content='What is the weather in sf?', id='6da9c792-36d4-417e-b401-7cf57bf1e726'), AIMessage(content='', response_metadata={'model': 'llama3.1', 'created_at': '2024-08-12T12:15:04.38621Z', 'message': {'role': 'assistant', 'content': '', 'tool_calls': [{'function': {'name': 'tavily_search_results_json', 'arguments': {'query': 'sf weather'}}}]}, 'done_reason': 'stop', 'done': True, 'total_duration': 31376375300, 'load_duration': 21196892100, 'prompt_eval_count': 249, 'prompt_eval_duration': 2830293000, 'eval_count': 22, 'eval_duration': 6540612000}, id='run-95302bdc-0fe1-4957-8a25-dae431504340-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'sf weather'}, 'id': '6a88d71a-5284-4886-b585-68783f051089', 'type': 'tool_call'}], usage_metadata={'input_tokens': 249, 'output_tokens': 22, 'total_tokens': 271}), ToolMessage(content='[{\'url\': \'https://www.weatherapi.com/\', \'content\': "{\'location\': {\'name\': \'San Franc

In [12]:
to_replay = states[-3]

In [13]:
to_replay

StateSnapshot(values={'messages': [HumanMessage(content='What is the weather in sf?', id='6da9c792-36d4-417e-b401-7cf57bf1e726'), AIMessage(content='', response_metadata={'model': 'llama3.1', 'created_at': '2024-08-12T12:15:04.38621Z', 'message': {'role': 'assistant', 'content': '', 'tool_calls': [{'function': {'name': 'tavily_search_results_json', 'arguments': {'query': 'sf weather'}}}]}, 'done_reason': 'stop', 'done': True, 'total_duration': 31376375300, 'load_duration': 21196892100, 'prompt_eval_count': 249, 'prompt_eval_duration': 2830293000, 'eval_count': 22, 'eval_duration': 6540612000}, id='run-95302bdc-0fe1-4957-8a25-dae431504340-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'sf weather'}, 'id': '6a88d71a-5284-4886-b585-68783f051089', 'type': 'tool_call'}], usage_metadata={'input_tokens': 249, 'output_tokens': 22, 'total_tokens': 271})]}, next=('action',), config={'configurable': {'thread_id': '1', 'thread_ts': '1ef58a48-81b4-679c-8001-8138eea2faa2'}}

In [14]:
for event in abot.graph.stream(None, to_replay.config):
    for k, v in event.items():
        print(v)

Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'sf weather'}, 'id': '6a88d71a-5284-4886-b585-68783f051089', 'type': 'tool_call'}
Back to the model!
{'messages': [ToolMessage(content='[{\'url\': \'https://www.weatherapi.com/\', \'content\': "{\'location\': {\'name\': \'San Francisco\', \'region\': \'California\', \'country\': \'United States of America\', \'lat\': 37.78, \'lon\': -122.42, \'tz_id\': \'America/Los_Angeles\', \'localtime_epoch\': 1723465031, \'localtime\': \'2024-08-12 05:17\'}, \'current\': {\'last_updated_epoch\': 1723464900, \'last_updated\': \'2024-08-12 05:15\', \'temp_c\': 13.2, \'temp_f\': 55.8, \'is_day\': 0, \'condition\': {\'text\': \'Partly Cloudy\', \'icon\': \'//cdn.weatherapi.com/weather/64x64/night/116.png\', \'code\': 1003}, \'wind_mph\': 9.2, \'wind_kph\': 14.8, \'wind_degree\': 240, \'wind_dir\': \'WSW\', \'pressure_mb\': 1014.0, \'pressure_in\': 29.95, \'precip_mm\': 0.0, \'precip_in\': 0.0, \'humidity\': 89, \'cloud\': 47, \'feelslik

# Go back in time and edit

In [ ]:
_id = to_replay.values['messages'][-1].tool_calls[0]['id']
to_replay.values['messages'][-1].tool_calls = [{'name': 'tavily_search_results_json',
  'args': {'query': 'current weather in LA, accuweather'},
  'id': _id}]

In [15]:
branch_state = abot.graph.update_state(to_replay.config, to_replay.values)

{'messages': [HumanMessage(content='What is the weather in sf?', id='6da9c792-36d4-417e-b401-7cf57bf1e726'), AIMessage(content='', response_metadata={'model': 'llama3.1', 'created_at': '2024-08-12T12:15:04.38621Z', 'message': {'role': 'assistant', 'content': '', 'tool_calls': [{'function': {'name': 'tavily_search_results_json', 'arguments': {'query': 'sf weather'}}}]}, 'done_reason': 'stop', 'done': True, 'total_duration': 31376375300, 'load_duration': 21196892100, 'prompt_eval_count': 249, 'prompt_eval_duration': 2830293000, 'eval_count': 22, 'eval_duration': 6540612000}, id='run-95302bdc-0fe1-4957-8a25-dae431504340-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'sf weather'}, 'id': '6a88d71a-5284-4886-b585-68783f051089', 'type': 'tool_call'}], usage_metadata={'input_tokens': 249, 'output_tokens': 22, 'total_tokens': 271})]}


In [16]:
for event in abot.graph.stream(None, branch_state):
    for k, v in event.items():
        if k != "__end__":
            print(v)

Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'sf weather'}, 'id': '6a88d71a-5284-4886-b585-68783f051089', 'type': 'tool_call'}
Back to the model!
{'messages': [ToolMessage(content='[{\'url\': \'https://www.weatherapi.com/\', \'content\': "{\'location\': {\'name\': \'San Francisco\', \'region\': \'California\', \'country\': \'United States of America\', \'lat\': 37.78, \'lon\': -122.42, \'tz_id\': \'America/Los_Angeles\', \'localtime_epoch\': 1723465101, \'localtime\': \'2024-08-12 05:18\'}, \'current\': {\'last_updated_epoch\': 1723464900, \'last_updated\': \'2024-08-12 05:15\', \'temp_c\': 13.2, \'temp_f\': 55.8, \'is_day\': 0, \'condition\': {\'text\': \'Partly Cloudy\', \'icon\': \'//cdn.weatherapi.com/weather/64x64/night/116.png\', \'code\': 1003}, \'wind_mph\': 9.2, \'wind_kph\': 14.8, \'wind_degree\': 240, \'wind_dir\': \'WSW\', \'pressure_mb\': 1014.0, \'pressure_in\': 29.95, \'precip_mm\': 0.0, \'precip_in\': 0.0, \'humidity\': 89, \'cloud\': 47, \'feelslik

# Add message to a state at a given time

In [17]:
to_replay

StateSnapshot(values={'messages': [HumanMessage(content='What is the weather in sf?', id='6da9c792-36d4-417e-b401-7cf57bf1e726'), AIMessage(content='', response_metadata={'model': 'llama3.1', 'created_at': '2024-08-12T12:15:04.38621Z', 'message': {'role': 'assistant', 'content': '', 'tool_calls': [{'function': {'name': 'tavily_search_results_json', 'arguments': {'query': 'sf weather'}}}]}, 'done_reason': 'stop', 'done': True, 'total_duration': 31376375300, 'load_duration': 21196892100, 'prompt_eval_count': 249, 'prompt_eval_duration': 2830293000, 'eval_count': 22, 'eval_duration': 6540612000}, id='run-95302bdc-0fe1-4957-8a25-dae431504340-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'sf weather'}, 'id': '6a88d71a-5284-4886-b585-68783f051089', 'type': 'tool_call'}], usage_metadata={'input_tokens': 249, 'output_tokens': 22, 'total_tokens': 271})]}, next=('action',), config={'configurable': {'thread_id': '1', 'thread_ts': '1ef58a48-81b4-679c-8001-8138eea2faa2'}}

In [18]:
_id = to_replay.values['messages'][-1].tool_calls[0]['id']

In [19]:
state_update = {"messages": [ToolMessage(
    tool_call_id=_id,
    name="tavily_search_results_json",
    content="54 degree celcius",
)]}

In [20]:
branch_and_add = abot.graph.update_state(
    to_replay.config, 
    state_update, 
    as_node="action")

In [21]:
for event in abot.graph.stream(None, branch_and_add):
    for k, v in event.items():
        print(v)

{'messages': [HumanMessage(content='What is the weather in sf?', id='6da9c792-36d4-417e-b401-7cf57bf1e726'), AIMessage(content='', response_metadata={'model': 'llama3.1', 'created_at': '2024-08-12T12:15:04.38621Z', 'message': {'role': 'assistant', 'content': '', 'tool_calls': [{'function': {'name': 'tavily_search_results_json', 'arguments': {'query': 'sf weather'}}}]}, 'done_reason': 'stop', 'done': True, 'total_duration': 31376375300, 'load_duration': 21196892100, 'prompt_eval_count': 249, 'prompt_eval_duration': 2830293000, 'eval_count': 22, 'eval_duration': 6540612000}, id='run-95302bdc-0fe1-4957-8a25-dae431504340-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'sf weather'}, 'id': '6a88d71a-5284-4886-b585-68783f051089', 'type': 'tool_call'}], usage_metadata={'input_tokens': 249, 'output_tokens': 22, 'total_tokens': 271}), ToolMessage(content='54 degree celcius', name='tavily_search_results_json', id='5c962916-50d7-406d-9bec-9990f5997ba5', tool_call_id='6a88